---
## Load The Data

In [1]:
import random
import utilities_tf1 as ut
import os
import pickle
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import time
%config InlineBackend.figure_format = 'svg'

In [2]:
# Load pickled data

# TODO: Fill this in based on where you saved the training and testing data

training_file = '../data/train.p'
validation_file= '../data/valid.p'
testing_file = '../data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [3]:
print ('X_train shape:', X_train.shape)
print ('X_valid shape:', X_valid.shape)
print ('X_test shape:', X_test.shape)

X_train shape: (34799, 32, 32, 3)
X_valid shape: (4410, 32, 32, 3)
X_test shape: (12630, 32, 32, 3)


## Pre-processing

In [4]:
X_train = ut.pre_process(X_train, channels=1)
X_valid = ut.pre_process(X_valid, channels=1)
X_test = ut.pre_process(X_test, channels=1)

In [5]:
print ('After Pre-processing:')
print ('X_train shape:', X_train.shape)
print ('X_valid shape:', X_valid.shape)
print ('X_test shape:', X_test.shape)

After Pre-processing:
X_train shape: (34799, 32, 32, 1)
X_valid shape: (4410, 32, 32, 1)
X_test shape: (12630, 32, 32, 1)


## Training Model (1): with original training/validation/test datasets

In [6]:
params = {
    'Input_channels': 1, 
    'n_classes': len(np.unique(y_train)), 
    'EPOCHS': 20, 
    'BATCH_SIZE': 128, 
    'rate': 0.001, 
    'training_operation': None,
    'accuracy_operation': None, 
    'x': None, 
    'y': None
}

Data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_valid': X_valid,
    'y_valid': y_valid,
    'X_test': X_test,
    'y_test': y_test
}

In [7]:
ut.Compile_Model(params)

In [8]:
ut.Train_and_Test_Model(params, Data)

Training...

EPOCH 5 ...
Training Accuracy = 0.881
Validation Accuracy = 0.702
Testing Accuracy = 0.705

EPOCH 10 ...
Training Accuracy = 0.951
Validation Accuracy = 0.748
Testing Accuracy = 0.752

EPOCH 15 ...
Training Accuracy = 0.964
Validation Accuracy = 0.754
Testing Accuracy = 0.746

EPOCH 20 ...
Training Accuracy = 0.972
Validation Accuracy = 0.758
Testing Accuracy = 0.762

Model saved


**Discussion**

the Training/evaluation results can be summaried in the table below: 

|Dataset|Accuracy (%)|
|-|-|
|Training set|97.2%|
|Validation set|75.8%|
|Test set|76.2%|

Discussion: 

- If the validation/test data comes from the same distribution as the training set, then there is a large variance problem and the algorithm is not generalizing well from the training set.
- However, if training data and the validation/test data come from  different distributions, there isn't necessarily a variance/overfitting problem. The problem might be that the validation/test set contains images that are more difficult to classify accurately.
- It's difficult to know whethere it is the abovementioned issue produces this approx 22% difference in accuracy between the training set and the validation/test set. 
- To resolve this issue, I define a new subset called `training-validation set`. This new subset has the same distribution as the training set, but it is not used for training the neural network.

## Training Model (2): with training-validation set

In [9]:
# training-validation data set
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                      test_size=0.2, random_state=0)

print ('X_train shape:', X_train.shape)
print ('X_valid shape:', X_valid.shape)

X_train shape: (27839, 32, 32, 1)
X_valid shape: (6960, 32, 32, 1)


In [10]:
params = {
    'Input_channels': 1, 
    'n_classes': len(np.unique(y_train)), 
    'EPOCHS': 20, 
    'BATCH_SIZE': 128, 
    'rate': 0.001, 
    'training_operation': None,
    'accuracy_operation': None, 
    'x': None, 
    'y': None
}

Data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_valid': X_valid,
    'y_valid': y_valid,
    'X_test': X_test,
    'y_test': y_test
}

In [11]:
ut.Compile_Model(params)

In [12]:
ut.Train_and_Test_Model(params, Data, train_validation=True)

Training...

EPOCH 5 ...
Training Accuracy = 0.837
Training-validation Accuracy = 0.798
Testing Accuracy = 0.650

EPOCH 10 ...
Training Accuracy = 0.925
Training-validation Accuracy = 0.874
Testing Accuracy = 0.701

EPOCH 15 ...
Training Accuracy = 0.966
Training-validation Accuracy = 0.910
Testing Accuracy = 0.726

EPOCH 20 ...
Training Accuracy = 0.977
Training-validation Accuracy = 0.920
Testing Accuracy = 0.734

Model saved


**Discussion**

the Training/evaluation results can be summaried in the table below: 

|Dataset|Accuracy (%)|
|-|-|
|Training set|97.7%|
|Training-Validation set|92.0%|
|Validation set (from previous run)|75.8%|
|Test set|73.4%|

## Conclusion

- From the analysis above, it can be concluded that the training set and validation/test set are very likely from diffierent distributions, which indicates that there might be a data dismaching issue. 
- To solve this problem, I have conducted a trail to add a portion of the test set data to the training set which is to enable the model to learn features of the test set. this has been illustrated in the main jupyter notebook "Traffic_Sign_Classifier_WX.ipynb" in the same folder. 